In [67]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM


df = pd.read_excel('/Users/martinwolf/Python/Superdepot Reporting/Data/df.xlsx')
df['Pick Datum'] = pd.to_datetime(df['B']).dt.date
#add 5 days to the end of the dataset



In [68]:
def DatenKundeaufbereiten(dfa):
    # DataFrame Picks zu Kunden
    dfapicks = dfa.groupby(['Lieferschein','Pick Datum','Kunde'],dropna =False)['Picks Gesamt'].sum().reset_index()
    dfapicksinCS = dfa.groupby(['Lieferschein','Pick Datum'])['Picks CS'].sum().reset_index()
    dfapicksinPAL = dfa.groupby(['Lieferschein','Pick Datum'])['Picks PAL'].sum().reset_index()
    dfapicksinOUT = dfa.groupby(['Lieferschein','Pick Datum'])['Picks OUT'].sum().reset_index()
    dfakunden = pd.merge(dfapicks, dfapicksinCS[['Lieferschein','Picks CS']],left_on='Lieferschein', right_on='Lieferschein',how='left')
    dfakunden = pd.merge(dfakunden, dfapicksinOUT[['Lieferschein','Picks OUT']],left_on='Lieferschein', right_on='Lieferschein',how='left')
    dfakunden = pd.merge(dfakunden, dfapicksinPAL[['Lieferschein','Picks PAL']],left_on='Lieferschein', right_on='Lieferschein',how='left')
    # DataFrame Picks zu Tag
    return dfakunden
def DatenPickTagaufbereiten(dfT):
    dfaPicksTag = dfT.groupby(['Pick Datum'])['Picks Gesamt'].sum().reset_index()
    dfaCStag = dfT.groupby(['Pick Datum'])['Picks CS'].sum().reset_index()   
    dfaPALtag = dfT.groupby(['Pick Datum'])['Picks PAL'].sum().reset_index()      
    dfaOUTtag = dfT.groupby(['Pick Datum'])['Picks OUT'].sum().reset_index()
    dfaZugriffe = pd.merge(dfaPicksTag, dfaCStag[['Pick Datum','Picks CS']],left_on='Pick Datum', right_on='Pick Datum',how='left')         
    dfaZugriffe = pd.merge(dfaZugriffe, dfaOUTtag[['Pick Datum','Picks OUT']],left_on='Pick Datum', right_on='Pick Datum',how='left')
    dfaZugriffe = pd.merge(dfaZugriffe, dfaPALtag[['Pick Datum','Picks PAL']],left_on='Pick Datum', right_on='Pick Datum',how='left')
    return dfaZugriffe

In [69]:
dfPicks = DatenPickTagaufbereiten(df)

df = dfPicks[['Pick Datum','Picks Gesamt']]
df.index = df['Pick Datum']
del df['Pick Datum']
test_days = 20

bitcoin = df.filter(['Picks Gesamt'])
#add next 5 workdays to the end of the dataset
bitcoin = bitcoin.append(pd.DataFrame({'Picks Gesamt': [0,0,0,0,0]}, index=pd.date_range(start=bitcoin.index[-1], periods=5, freq='D')))
#bitcoin = bitcoin.append(pd.DataFrame({'Picks Gesamt': [0,0,0,0,0]}, index=[bitcoin.index[-1] + pd.DateOffset(days=x) for x in range(1,6)]))



/var/folders/0j/czkb_cmx12d_xchj2bf8bptc0000gn/T/ipykernel_6175/1682030100.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bitcoin = bitcoin.append(pd.DataFrame({'Picks Gesamt': [0,0,0,0,0]}, index=pd.date_range(start=bitcoin.index[-1], periods=5, freq='D')))


In [71]:
bitcoin.index = bitcoin['my_dates'].dt.day_name()
bitcoin

KeyError: 'my_dates'